In [53]:
c.execute("""DROP TABLE Votes""")
c.execute("""DROP TABLE MPs""")
c.execute("""DROP TABLE Parties""")
c.execute("""DROP TABLE Motions""")
c.execute("""DROP TABLE Affiliations""")

conn.commit()

In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('database.db')

c = conn.cursor()

In [54]:
# MPs table tiesiog sarasas visu MPs vardu ir ID
c.execute("""
CREATE TABLE IF NOT EXISTS MPs
(Name TEXT,
 MP_id INTEGER PRIMARY KEY) 
        """)

# Sarasas visu balsavimu
c.execute("""
CREATE TABLE IF NOT EXISTS Motions
(Term_id INTEGER,
 Session_id INTEGER,
 Meeting_id INTEGER,
 Motion_id INTEGER PRIMARY KEY,
 Motion_type TEXT,
 Motion TEXT)
        """)

# Sarasas kiekvieno balso (kiekvienas balsas yra unikali kombinacija motion_id ir mp_id)
c.execute("""
CREATE TABLE IF NOT EXISTS Votes
(MP_id INTEGER NOT NULL,
 Motion_id INTEGER NOT NULL,
 Vote INTEGER,
 PRIMARY KEY (MP_id, Motion_id)) 
        """)

# Sarasas partiju
c.execute("""
CREATE TABLE IF NOT EXISTS Parties
(Name TEXT PRIMARY KEY)
""")

# Sarasas afiliaciju partijoms; atskirtas nuo MPs table, nes skirtingose kadencijose tas pats MP gali priklausyti skirtingoms partijoms
c.execute("""
CREATE TABLE IF NOT EXISTS Affiliations
(Party_id INTEGER,
Term_id INTEGER,
MP_id INTEGER,
PRIMARY KEY (Term_id, MP_id))""")

conn.commit()


In [51]:
c.close()
conn.close()

Scrapinimo logika:

1. Kadencijos -> sesijos -> posedziai (straightforward).
2. Posedzio eigoje perrenki klausimus, jeigu klausimas vadinasi "klausimu grupe <...>", posedzio darbotvarkeje susirandi klausimu pavadinimus.
3. Klausimo eigoje patikrini, ar klausimas turi skilti "Balsavimas" - jei taip, su tuo id patikrini balsavimo rezultatus ir surenki.

In [78]:
# uzsipildom MPs, Parties ir Affiliations tables

kadencijos_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_kadencijos'
kadencijos_page = requests.get(kadencijos_url)
kadencijos_soup = BeautifulSoup(kadencijos_page.content, 'html5lib')
kadencijos = kadencijos_soup.find_all('seimokadencija')
kadencijos_ids = [kadencija.attrs['kadencijos_id'] for kadencija in kadencijos]

# pirmos 4 kadencijos inconsistently savo duomenis sudejusios, tai nuo penktos pradejau rinkti
for kadencijos_id in kadencijos_ids[4:]:
    
    kadencijos_id_int = int(kadencijos_id)
    
    seimo_nariai_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_nariai?kadencijos_id=' + kadencijos_id
    seimo_nariai_page = requests.get(seimo_nariai_url)
    seimo_nariai_soup = BeautifulSoup(seimo_nariai_page.content, 'html5lib')
    seimo_nariai = seimo_nariai_soup.find_all('seimonarys')
    
    for seimo_narys in seimo_nariai:
        
        mp_id_int = int(seimo_narys.attrs['asmens_id'])
        name = seimo_narys.attrs['vardas'] + ' ' + seimo_narys.attrs['pavardė']
        partija = seimo_narys.attrs['iškėlusi_partija']
        
        c.execute("""
        INSERT OR IGNORE INTO Parties
        (Name)
        VALUES (?)
        """, (partija,))
        
        c.execute("""
        SELECT rowid FROM Parties
        WHERE Name = ?""", (partija,))

        party_id = c.fetchall()[0][0]
        
        c.execute("""
        INSERT OR IGNORE INTO Affiliations
        (Party_id, Term_id, MP_id)
        VALUES (?,?,?)
        """,
        (party_id, kadencijos_id_int, mp_id_int))
        
        c.execute("""
        INSERT OR IGNORE INTO MPs
        (Name, MP_id)
        VALUES (?,?)""", (name, mp_id_int))
        
conn.commit()
        
        
    

In [80]:
balsai_int = {
    'Prieš': -1,
    'Už': 1,
    'Susilaikė': 0,
    '': None
}

# kadencija -> sesija -> posedis -> klausimas -> balsavimas uz klausima
kadencijos_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_kadencijos'
kadencijos_page = requests.get(kadencijos_url)
kadencijos_soup = BeautifulSoup(kadencijos_page.content, 'html5lib')
kadencijos = kadencijos_soup.find_all('seimokadencija')
kadencijos_ids = [kadencija.attrs['kadencijos_id'] for kadencija in kadencijos]

for kadencijos_id in kadencijos_ids[8:]:
    
    sesijos_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_sesijos?kadencijos_id=' + kadencijos_id
    sesijos_page = requests.get(sesijos_url)
    sesijos_soup = BeautifulSoup(sesijos_page.content, 'html5lib')
    sesijos = sesijos_soup.find_all('seimosesija')
    sesijos_ids = [sesija.attrs['sesijos_id'] for sesija in sesijos]
    
    for sesijos_id in sesijos_ids:
        
        posedziai_url = 'http://apps.lrs.lt/sip/p2b.ad_seimo_posedziai?sesijos_id=' + sesijos_id
        posedziai_page = requests.get(posedziai_url)
        posedziai_soup = BeautifulSoup(posedziai_page.content, 'html5lib')
        posedziai = posedziai_soup.find_all('seimoposėdis')
        posedziai_ids = [posedis.attrs['posėdžio_id'] for posedis in posedziai]
        
        for posedis_id in posedziai_ids:
            
            posedis_url = 'http://apps.lrs.lt/sip/p2b.ad_sp_eiga?posedzio_id=' + posedis_id 
            posedis_page = requests.get(posedis_url)
            posedis_soup = BeautifulSoup(posedis_page.content, 'html5lib')
            klausimai = posedis_soup.find_all('eigosklausimas')
            klausimai_ids = [klausimas.attrs['svarstomo_klausimo_id'] for klausimas in klausimai]
            klausimai_names = [klausimas.attrs['pavadinimas'] for klausimas in klausimai]
            klausimai_types = [klausimas.attrs['stadija'] for klausimas in klausimai]
            
            for i in range(len(klausimai_ids)):
                
                klausimas_id, klausimas_name, klausimas_type = klausimai_ids[i], klausimai_names[i], klausimai_types[i]
                
                balsavimas_url = 'http://apps.lrs.lt/sip/p2b.ad_sp_balsavimo_rezultatai?balsavimo_id=' + klausimas_id
                balsavimas_page = requests.get(balsavimas_url)
                balsavimas_soup = BeautifulSoup(balsavimas_page.content, 'html5lib')
                balsai = balsavimas_soup.find_all('individualusbalsavimorezultatas')
                
                if balsai != []:
                    
                    c.execute("""
                    INSERT OR IGNORE INTO Motions
                    (Term_id, Session_id, Meeting_id, Motion_id, Motion_type, Motion)
                    VALUES (?,?,?,?,?,?)""", (int(kadencijos_id), int(sesijos_id), int(posedis_id), int(klausimas_id), klausimas_type, klausimas_name))
                
                    
                    for balsas in balsai:
                        
                        balsas_type = balsas.attrs['kaip_balsavo']
                        mp_id = int(balsas.attrs['asmens_id'])
                        klausimas_id_int = int(klausimas_id)
                        balsas_int = balsai_int[balsas_type]
                        
                        c.execute("""
                        INSERT OR IGNORE INTO Votes
                        (MP_id, Motion_id, Vote)
                        VALUES (?,?,?)
                        """, (mp_id, klausimas_id_int, balsas_int))
                        
                        conn.commit()
                        
                conn.commit()
                        
                        
                        
                        
            
        


In [56]:
c.execute("""
SELECT * FROM Votes""")

ans = c.fetchall()

print(ans)

# 6 80 -500186 -3664

[(47852, -3168, None), (7229, -3168, None), (45771, -3168, None), (7208, -3168, None), (47958, -3168, None), (47220, -3168, None), (48114, -3168, None), (23552, -3168, None), (23612, -3168, None), (47834, -3168, None), (26656, -3168, None), (16689, -3168, None), (7326, -3168, None), (22, -3168, None), (47598, -3168, None), (47838, -3168, None), (265, -3168, None), (47856, -3168, None), (47850, -3168, None), (7194, -3168, None), (7260, -3168, None), (37157, -3168, None), (7232, -3168, None), (47860, -3168, None), (7444, -3168, None), (31, -3168, None), (47844, -3168, None), (47837, -3168, None), (7207, -3168, None), (34972, -3168, None), (47828, -3168, None), (23703, -3168, None), (47849, -3168, None), (47841, -3168, None), (47840, -3168, None), (47842, -3168, None), (47859, -3168, None), (155, -3168, None), (47867, -3168, None), (7199, -3168, None), (34428, -3168, None), (47843, -3168, None), (47832, -3168, None), (38456, -3168, None), (104, -3168, None), (33749, -3168, None), (47833, 

In [16]:
mp_id

47865

In [40]:
response = requests.get('http://apps.lrs.lt/sip/p2b.ad_seimo_posedziai?sesijos_id=1')
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html5lib')
soup

<!--?xml version="1.0" encoding="UTF-8"?--><html><head></head><body><seimoinformacija licencija="CC BY 4.0" pavadinimas="Seimo posėdžiai" suformavimo_laikas="2023-07-23 15:51:03" tiekėjas="Seimo kanceliarija" šaltinis="www.lrs.lt">
<seimosesija sesijos_id="1">
<seimoposėdis numeris="-1" pabaiga="1990-05-19 00:00" posėdžio_id="-499933" pradžia="1990-05-19 00:00" tipas="Rytinis posėdis">
<stenograma stenogramos_nuoroda="https://e-seimas.lrs.lt/portal/legalAct/lt/TAK/TAIS.251027">
</stenograma></seimoposėdis>
<seimoposėdis numeris="-1" pabaiga="1990-05-16 00:00" posėdžio_id="-499936" pradžia="1990-05-16 00:00" tipas="Uždaras posėdis">
<stenograma stenogramos_nuoroda="https://e-seimas.lrs.lt/portal/legalAct/lt/TAK/TAIS.251746">
</stenograma></seimoposėdis>
<seimoposėdis numeris="-1" pabaiga="1990-03-12 00:00" posėdžio_id="-499995" pradžia="1990-03-12 00:00" tipas="Uždaras posėdis">
<stenograma stenogramos_nuoroda="https://e-seimas.lrs.lt/portal/legalAct/lt/TAK/TAIS.251750">
</stenograma></